In [1]:
!curl -o "ml-latest.zip" "http://files.grouplens.org/datasets/movielens/ml-latest.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  264M  100  264M    0     0  60.8M      0  0:00:04  0:00:04 --:--:-- 60.8M


In [2]:
!curl -o "keyvec.zip" "http://vectors.nlpl.eu/repository/20/6.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  605M  100  605M    0     0  17.2M      0  0:00:35  0:00:35 --:--:-- 22.0M


In [8]:
%%bash
unzip keyvec.zip
unzip ml-latest.zip

Archive:  ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [22]:
!rm ml-latest/production.csv

In [13]:
%%bash 
echo '{"username":"anirudhnarsipur","key":"6cc2804ccd69dd37b0b72de821ef47d3"}' > ~/.kaggle/kaggle.json 

In [14]:
!kaggle datasets download -d danielgrijalvas/movies

  0% 0.00/369k [00:00<?, ?B/s]
100% 369k/369k [00:00<00:00, 58.5MB/s]


In [24]:
!unzip movies.zip

Archive:  movies.zip
  inflating: movies.csv              


In [35]:
import os 
import re
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from gensim.test.utils import datapath


In [23]:
!ls

keyvec.zip  ml-latest	   model.bin  movies.zip  sample_data
meta.json   ml-latest.zip  model.txt  README


In [17]:
data_dir = 'ml-latest/'
data_files = os.listdir('ml-latest/')
print(data_files)

['movies.csv', 'tags.csv', 'production.csv', 'README.txt', 'ratings.csv', 'genome-scores.csv', 'links.csv', 'genome-tags.csv']


In [18]:

movies = pd.read_csv(data_dir+"movies.csv")
metadata = pd.read_csv(data_dir+'genome-tags.csv')
ratings = pd.read_csv(data_dir + 'ratings.csv')


In [25]:
produc = pd.read_csv("movies.csv",encoding='latin')

In [26]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
metadata.sample(5)

,tagId,tag
465,466,great cinematography
47,48,alone in the world
1091,1092,visually stunning
885,886,sci fi
948,949,spanish civil war


In [28]:
produc.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [29]:
movies["name"] = movies["title"].apply(lambda s:re.sub(r" ?\([^)]+\)", "",s) )
movies.head()

,movieId,title,genres,name
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II


In [30]:
df = pd.merge(movies,produc,on="name")
df.head()

,movieId,title,genres,name,budget,company,country,director,genre,gross,rating,released,runtime,score,star,votes,writer,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,30000000.0,Pixar Animation Studios,USA,John Lasseter,Animation,191796233.0,G,1995-11-22,81,8.3,Tom Hanks,694113,John Lasseter,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,50000000.0,TriStar Pictures,USA,Joe Johnston,Action,100475249.0,PG,1995-12-15,104,6.9,Robin Williams,232339,Jonathan Hensleigh,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,25000000.0,Lancaster Gate,USA,Howard Deutch,Comedy,69870000.0,PG-13,1995-12-22,101,6.6,Walter Matthau,20718,Mark Steven Johnson,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,16000000.0,Twentieth Century Fox Film Corporation,USA,Forest Whitaker,Comedy,67052156.0,R,1995-12-22,124,5.7,Whitney Houston,7977,Terry McMillan,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,30000000.0,Sandollar Productions,USA,Charles Shyer,Comedy,76594000.0,PG,1995-12-08,106,5.9,Steve Martin,28658,Albert Hackett,1995


In [31]:
df[~df["country"].isin(["USA","UK"])]["country"].value_counts()

France                            198
Canada                            106
Germany                            73
Japan                              57
Australia                          55
Italy                              45
India                              37
Spain                              35
Ireland                            26
South Korea                        25
Hong Kong                          24
Mexico                             19
China                              18
New Zealand                        16
Denmark                            16
Sweden                             15
Russia                             14
Taiwan                              9
Czech Republic                      9
Belgium                             7
Iran                                7
Argentina                           7
Switzerland                         6
Brazil                              6
Netherlands                         6
Austria                             6
Norway      

In [32]:
ratings['userId'].unique().size

283228

In [38]:
wv = KeyedVectors.load_word2vec_format("model.bin", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [47]:
wv["007"].shape

(300,)

In [200]:
ratings_sample = ratings[ratings["userId"]<=50]

In [61]:
foreign = df[~df["country"].isin(["USA","UK"])]

In [62]:
foreign.head()

,movieId,title,genres,name,budget,company,country,director,genre,gross,rating,released,runtime,score,star,votes,writer,year
13,7,Sabrina (1995),Comedy|Romance,Sabrina,58000000.0,Constellation Entertainment,Germany,Sydney Pollack,Comedy,53672080.0,PG,1995-12-15,127,6.3,Harrison Ford,31451,Samuel A. Taylor,1995
14,915,Sabrina (1954),Comedy|Romance,Sabrina,58000000.0,Constellation Entertainment,Germany,Sydney Pollack,Comedy,53672080.0,PG,1995-12-15,127,6.3,Harrison Ford,31451,Samuel A. Taylor,1995
22,16,Casino (1995),Crime|Drama,Casino,52000000.0,Universal Pictures,France,Martin Scorsese,Crime,42438300.0,R,1995-11-22,178,8.2,Robert De Niro,367532,Nicholas Pileggi,1995
31,23,Assassins (1995),Action|Crime|Thriller,Assassins,50000000.0,Warner Bros.,France,Richard Donner,Action,30306268.0,R,1995-10-06,132,6.3,Sylvester Stallone,67317,Lilly Wachowski,1995
40,33,Wings of Courage (1995),Adventure|Romance|IMAX,Wings of Courage,0.0,Iwerks Entertainment,France,Jean-Jacques Annaud,Adventure,15054389.0,G,1996-09-18,50,6.5,Craig Sheffer,537,Alain Godard,1995


In [72]:
genome = pd.read_csv(data_dir+"genome-scores.csv")

In [126]:
genome[genome["movieId"].isin(np.array([1,2,3]))]["tagId"]

0          1
1          2
2          3
3          4
4          5
        ... 
3379    1124
3380    1125
3381    1126
3382    1127
3383    1128
Name: tagId, Length: 3384, dtype: int64

In [163]:
#TO DO: Account for tag relevance
def get_tags_relevance(mid):
  all_tags = genome[genome["movieId"]==mid]
  rel = -np.partition(-np.array(all_tags["relevance"]),20)[:20]
  temp = all_tags[all_tags["relevance"].isin(rel)]["tagId"][:20].tolist()
  return metadata[metadata["tagId"].isin(temp)]["tag"].tolist()

def get_vecs(tags):
  ls = []
  for tag in tags:
    for word in [i.lower() for i in tag.split()]:
      try:
        ls.append(wv[tag])
      except:
          pass
  return np.array(ls)

def id_vec(id):
  return get_vecs(get_tags_relevance(id))



In [157]:
foreign_vecs = pd.DataFrame()

In [164]:
foreign_vecs["movieId"] = foreign["movieId"]
foreign_ids = foreign_vecs["movieId"].tolist()
all_vecs = [id_vec(mid) for mid in foreign_vecs["movieId"].tolist()]

In [221]:
def remove_null(vec):
  ls =  [] 
  for i in vec:
    if i.size == 0:
      ls.append(np.zeros((1,300)))
    else:
      ls.append(i)
  return np.array(ls)


In [217]:
foreign_vecs["movie_vec"] = [remove_null(vec) for vec in all_vecs]
foreign_vecs.head()

,movieId,movie_vec
13,7,"[[0.093107, -0.01204, 0.000203, 0.079919, -0.0..."
14,915,"[[-0.012918, -0.050051, 0.030364, -0.145345, 0..."
22,16,"[[0.167327, -0.005989, -0.046007, 0.152078, 0...."
31,23,"[[-0.014575, 0.033459, 0.018808, 0.097272, 0.0..."
40,33,"[[0.026536, -0.04169, 0.066246, -0.084588, 0.0..."


In [201]:
ratings_sample['movieId'] = ratings_sample["movieId"].apply(lambda x: str(x))
user_movies = ratings_sample.groupby(ratings_sample["userId"]).agg({"movieId": lambda x: ','.join(x)})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [225]:
def get_ls_vecs(ls_ids):
  ls_ids = [int(i) for i in ls_ids.split(',')]
  all_vecs = [remove_null(id_vec(mid)) for mid in ls_ids]
  out = []
  for vec in all_vecs:
    if vec.size == 0:
      continue 
    else:
        out.append(vec)
  return np.vstack(out)


In [226]:
user_movies["user_vector"] = user_movies["movieId"].apply(get_ls_vecs)

In [230]:
user_movies.head()

,movieId,user_vector
userId,,
1,"307,481,1091,1257,1449,1590,1591,2134,2478,284...","[[0.016626, 0.06113, 0.06345, -0.008968, -0.01..."
2,"170,849,1186,1235,1244,1296,1663,1962,2108,224...","[[0.044119, -0.048573, 0.021254, 0.052899, -0...."
3,"640,828,960,1221,1321,1645,1825,1985,2024,2028...","[[0.106495, -0.017905, 0.043308, 0.015604, -0...."
4,"1,2,5,6,10,11,16,19,20,23,25,31,32,34,36,39,41...","[[-0.012086, 0.083213, -0.07851, 0.033313, 0.0..."
5,"47,50,293,296,318,527,728,778,858,1147,1186,11...","[[-0.00103, -0.024518, -0.046511, -0.010788, 0..."


In [240]:
def clean(x): 
  if x.size == 0: 
    return np.zeros((1,300)) 
  else: 
    return x
foreign["movie_vec"] = foreign_vecs["movie_vec"].apply(clean)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [241]:
foreign.head()

,movieId,title,genres,name,budget,company,country,director,genre,gross,rating,released,runtime,score,star,votes,writer,year,movie_vec
13,7,Sabrina (1995),Comedy|Romance,Sabrina,58000000.0,Constellation Entertainment,Germany,Sydney Pollack,Comedy,53672080.0,PG,1995-12-15,127,6.3,Harrison Ford,31451,Samuel A. Taylor,1995,"[[0.093107, -0.01204, 0.000203, 0.079919, -0.0..."
14,915,Sabrina (1954),Comedy|Romance,Sabrina,58000000.0,Constellation Entertainment,Germany,Sydney Pollack,Comedy,53672080.0,PG,1995-12-15,127,6.3,Harrison Ford,31451,Samuel A. Taylor,1995,"[[-0.012918, -0.050051, 0.030364, -0.145345, 0..."
22,16,Casino (1995),Crime|Drama,Casino,52000000.0,Universal Pictures,France,Martin Scorsese,Crime,42438300.0,R,1995-11-22,178,8.2,Robert De Niro,367532,Nicholas Pileggi,1995,"[[0.167327, -0.005989, -0.046007, 0.152078, 0...."
31,23,Assassins (1995),Action|Crime|Thriller,Assassins,50000000.0,Warner Bros.,France,Richard Donner,Action,30306268.0,R,1995-10-06,132,6.3,Sylvester Stallone,67317,Lilly Wachowski,1995,"[[-0.014575, 0.033459, 0.018808, 0.097272, 0.0..."
40,33,Wings of Courage (1995),Adventure|Romance|IMAX,Wings of Courage,0.0,Iwerks Entertainment,France,Jean-Jacques Annaud,Adventure,15054389.0,G,1996-09-18,50,6.5,Craig Sheffer,537,Alain Godard,1995,"[[0.026536, -0.04169, 0.066246, -0.084588, 0.0..."


In [303]:
def cosine_sim(a,b):
  try:
    dotp = np.matmul(a,b.T)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    if norma == 0 or normb ==0:
      return 0
    else:
      return np.linalg.norm(dotp/(norma * normb))
  except:
    return 0 

In [333]:

def get_ranking(vec):
  return [cosine_sim(vec,fvec) for fvec in foreign["movie_vec"]]


In [340]:
def get_movies(ls_movies):
  mids = [int(mid) for mid in ls_movies.split(',')]
  return movies[movies["movieId"].isin(mids)][["title","genres"]]
def get_rank_list(num):
  tst = user_movies['user_vector'].iloc[num]
  foreign["Scores"] = get_ranking(tst)
  print(get_movies(user_movies["movieId"].iloc[num]))
  print("----------------------------------------------")
  print(foreign.sort_values("Scores",ascending=False)[["name","genres"]].head(10))

In [345]:
get_rank_list(5)


                                                 title                                           genres
9                                     GoldenEye (1995)                        Action|Adventure|Thriller
20                                   Get Shorty (1995)                            Comedy|Crime|Thriller
31           Twelve Monkeys (a.k.a. 12 Monkeys) (1995)                          Mystery|Sci-Fi|Thriller
46                         Seven (a.k.a. Se7en) (1995)                                 Mystery|Thriller
49                          Usual Suspects, The (1995)                           Crime|Mystery|Thriller
109                                 Taxi Driver (1976)                             Crime|Drama|Thriller
139                               Birdcage, The (1996)                                           Comedy
151                              Batman Forever (1995)                    Action|Adventure|Comedy|Crime
159                                Crimson Tide (1995)          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
